In [1]:
import numpy as np
from astropy import units as u
from astropy import constants as const
from astropy.io import ascii
from scipy import interpolate
from bit_tools.mission_tools import Mission
import bit_tools as bt

from mag_converter import *

/home/veome/anaconda3/lib/python3.8/site-packages/pyds9.py:94: UserWarning: Can't locate DS9 executable. Please add the DS9 directory to your PATH and try again.
  warnings.warn(ds9_warning)


In [2]:
magcut = 26.5
DIRECTORY = '/mnt/44868D0F868D02A2/Users/veome/Google Drive/Academics/Senior Thesis/Code/Catalog/'

In [3]:
cosmos = np.genfromtxt(DIRECTORY+'acs_cosmos1361.gcat_ascii', usecols=range(0,11), invalid_raise=False)

<ipython-input-3-0a010cf2ec54>:1: ConversionWarning: Some errors were detected !
    Line #2617 (got 10 columns instead of 11)
  cosmos = np.genfromtxt(DIRECTORY+'acs_cosmos1361.gcat_ascii', usecols=range(0,11), invalid_raise=False)


In [4]:
# Get HST F814W Band from bit_tools/bit_tools/data/missions/hst/bandpass/
hst = Mission("hst")
band = hst.get_bandpass("f814w")
f814w_band_response = band.r_lam
f814w_pivot = band.lam_pivot * (u.nm.to(u.AA))


# Change wavelength axis units from nm to Angstrom
# x = band.get_wavelengths() 
x = np.arange(300, 1200, 1)
f814w = interpolate.interp1d(x*10, f814w_band_response(x), fill_value="extrapolate")

In [5]:
# Get SuperBIT Shape Band from bit_tools/bit_tools/data/missions/superbit/bandpass/
# bit = Mission("superbit19")
# r_shape = bit.get_bandpass("lum").r_lam

bit = Mission("superbit")
band = bit.get_bandpass("shape")
r_shape = band.r_lam
shape_pivot = band.lam_pivot * (u.nm.to(u.AA))



# Change wavelength axis units from nm to Angstrom
# x = bit.get_bandpass("shape").get_wavelengths()
x = np.arange(300, 1200, 1)
sbit_shape = interpolate.interp1d(x*10, r_shape(x), fill_value="extrapolate")

In [6]:
# Elliptical Template
elliptical_template = np.genfromtxt('galaxy_spectra_kinney96/elliptical_template.ascii', usecols=range(2), invalid_raise=False)

In [7]:
# Starburst Template
starburst_all = [[], [], [], [], [], []]
for i in range(1, 7):
    starburst_all[i-1] = np.genfromtxt('galaxy_spectra_kinney96/starb%i_template.ascii' %(i), usecols=range(2), invalid_raise=False)

starburst_template = np.mean( np.array([starburst_all[0], starburst_all[1], starburst_all[2], starburst_all[3], starburst_all[4], starburst_all[5]]), axis=0 )

In [8]:
# Initialization
cat = cosmos[cosmos[:,2] > 0] # redshift cut
cat = cat[cat[:,5] <= magcut] # mag cut
length = cat.shape[0]
width = cat.shape[1]

print("Original Catalog Size: %i"%cosmos.shape[0])
print("New Catalog Size: %i"%length)

Original Catalog Size: 697594
New Catalog Size: 488143


In [9]:
# Mag conversion assuming both Elliptical and Starburst SEDs
z = cat[:,2]
mag_obs = cat[:,5]
super_e_mag = np.zeros(length)
super_starb_mag = np.zeros(length)

for i in range(length):
    if (i%(10000) == 0):
        percentage = int((i/length)*100)
        print("Object {} of {} ({}%)".format(i, length, percentage))
        
    # ST Mag Conversion    
#     super_e_mag[i], super_starb_mag[i] = stmag_conversion(mag_obs[i], z[i], f814w, sbit_shape, elliptical_template, starburst_template)  
    
    # AB Mag Conversion    
#     super_e_mag[i], super_starb_mag[i] = abmag_conversion(mag_obs[i], z[i], f814w, sbit_shape, elliptical_template, starburst_template, src_pivot=f814w_pivot, target_pivot=shape_pivot) 
    
    # ST to AB Mag Conversion (Faster than just AB Mag)
    st_mag_obs = bt.converters.magab_to_magst(mag_obs[i], f814w_pivot * u.AA.to(u.nm))
    
    super_e_mag[i], super_starb_mag[i] = stmag_conversion(st_mag_obs, z[i], f814w, sbit_shape, elliptical_template, starburst_template)
    
    super_e_mag[i] = bt.converters.magst_to_magab(super_e_mag[i], shape_pivot * u.AA.to(u.nm))
    super_starb_mag[i] = bt.converters.magst_to_magab(super_starb_mag[i], shape_pivot * u.AA.to(u.nm))
    
print("Conversion complete!")

Object 0 of 488143 (0%)
Object 10000 of 488143 (2%)
Object 20000 of 488143 (4%)
Object 30000 of 488143 (6%)
Object 40000 of 488143 (8%)
Object 50000 of 488143 (10%)
Object 60000 of 488143 (12%)
Object 70000 of 488143 (14%)
Object 80000 of 488143 (16%)
Object 90000 of 488143 (18%)
Object 100000 of 488143 (20%)
Object 110000 of 488143 (22%)
Object 120000 of 488143 (24%)
Object 130000 of 488143 (26%)
Object 140000 of 488143 (28%)
Object 150000 of 488143 (30%)
Object 160000 of 488143 (32%)
Object 170000 of 488143 (34%)
Object 180000 of 488143 (36%)
Object 190000 of 488143 (38%)
Object 200000 of 488143 (40%)
Object 210000 of 488143 (43%)
Object 220000 of 488143 (45%)
Object 230000 of 488143 (47%)
Object 240000 of 488143 (49%)
Object 250000 of 488143 (51%)
Object 260000 of 488143 (53%)
Object 270000 of 488143 (55%)
Object 280000 of 488143 (57%)
Object 290000 of 488143 (59%)
Object 300000 of 488143 (61%)
Object 310000 of 488143 (63%)
Object 320000 of 488143 (65%)
Object 330000 of 488143 (67%)

In [10]:
# Load the extracted data from the plot. [:,0] is the redshift, [:,1] is the fraction
disk_frac = np.genfromtxt('gal_types/disks.csv', invalid_raise=False,delimiter=',')
ellip_frac = np.genfromtxt('gal_types/ellips.csv', invalid_raise=False,delimiter=',')
pec_frac = np.genfromtxt('gal_types/pecs.csv', invalid_raise=False,delimiter=',')

# Create interpolation objects for the fractions
disk_frac = interpolate.interp1d(disk_frac[:,0],disk_frac[:,1],fill_value=(disk_frac[:,1][0],disk_frac[:,1][-1]),bounds_error=False)
ellip_frac = interpolate.interp1d(ellip_frac[:,0],ellip_frac[:,1],fill_value=(ellip_frac[:,1][0],ellip_frac[:,1][-1]),bounds_error=False)
pec_frac = interpolate.interp1d(pec_frac[:,0],pec_frac[:,1],fill_value=(pec_frac[:,1][0],pec_frac[:,1][-1]),bounds_error=False)

In [11]:
# Determine whether or not the current source should be an elliptical
def is_elliptical(z):
    rand = np.random.uniform(0,ellip_frac(z)+disk_frac(z) + pec_frac(z))
    if rand <= disk_frac(z):
        return True
    if disk_frac(z)<rand <= ellip_frac(z)+disk_frac(z):
        return True
    if ellip_frac(z)+disk_frac(z) < rand <= ellip_frac(z)+disk_frac(z) + pec_frac(z):
        return False

In [12]:
# Generate a z-dependent selection of Elliptical and Starburst mags
super_mixed_mag = np.zeros(length)
for i in range(length):
    z = cat[i,2]
    if (is_elliptical(z)):
        super_mixed_mag[i] = super_e_mag[i]
    else:
        super_mixed_mag[i] = super_starb_mag[i]

In [13]:
all_data = np.zeros((length,width+3))
all_data[:,0:width] = cat
all_data[:,width] =  np.around(super_e_mag, decimals=5)
all_data[:,width+1] = np.around(super_starb_mag, decimals=5)
all_data[:,width+2] = np.around(super_mixed_mag, decimals=5)

In [14]:
headers = np.array(['# RA [deg]', 'Dec [deg]', 'Redshift', 'gamma1', 'gamma2', 'Magnitude', 'Weight', 'Size', 'UncorSize', 'FWHM', 'SNR_HST', 'SuperBIT E Mag', 'SuperBIT Starburst Mag', 'SuperBIT Mixed Mag'])

In [15]:
fname = 'superbit_cosmos1361_shape_abmag26.5.gcat_ascii'
ascii.write(all_data, fname, delimiter=' ', format='fixed_width', names=headers, overwrite=True)
print("Catalog written to {}".format(fname))

Catalog written to superbit_cosmos1361_shape_abmag26.5.gcat_ascii


In [16]:
np.amax(super_e_mag)

28.175963954507015